In [ ]:
!pip install pymongo
!pip install pandas

  Using cached BeautifulSoup-3.2.2.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [22 lines of output]
      Traceback (most recent call last):
        File "d:\Р’РЈР—\РёРЅР¶РµРЅРёСЂРµРЅРі РґР°РЅРЅС‹С…\github\DataEngineeringHomework\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "d:\Р’РЈР—\РёРЅР¶РµРЅРёСЂРµРЅРі РґР°РЅРЅС‹С…\github\DataEngineeringHomework\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "d:\Р’РЈР—\РёРЅР¶РµРЅРёСЂРµРЅРі РґР°РЅРЅС‹С…\github\DataEngineeringHomework\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in get_requires_for_build_wheel
          return hook(config_settings)
                 ^^^^^^^^^^^^^^^

***Вариант №36***

Задание №1

In [51]:
import pandas as pd
from pymongo import MongoClient
import json

# Подключение к MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["task_1"]
collection = db["task_1"]

# Чтение данных из файла
file_path = "task_1_item.text"
with open(file_path, "r", encoding="utf-8") as file:
    data = file.read()

# Разделение данных на записи
records = data.split("=====")
json_records = []
for record in records:
    if record.strip():
        fields = record.strip().split("\n")
        json_record = {}
        for field in fields:
            key, value = field.split("::")
            json_record[key.strip()] = value.strip()
        json_records.append(json_record)

# Запись данных в MongoDB
collection.delete_many({})  # Очистка коллекции перед записью новых данных
collection.insert_many(json_records)

# Запрос 1: Первые 10 записей, отсортированных по убыванию salary
result1 = list(collection.find({}, {"_id": 0}).sort("salary", -1).limit(10))
df = pd.DataFrame.from_dict(result1)
df.to_json("1_1.json", orient="records", indent=1, default_handler=str, force_ascii=False)
print("\nЗапрос 1:", json.dumps(result1, ensure_ascii=False, indent=4))

# Запрос 2: Первые 15 записей с age < 30, отсортированных по убыванию salary
result2 = list(
    collection.find({"age": {"$lt": "30"}}, {"_id": 0}).sort("salary", -1).limit(15)
)
df = pd.DataFrame.from_dict(result2)
df.to_json("1_2.json", orient="records", indent=1, default_handler=str, force_ascii=False)
print("\nЗапрос 2:", json.dumps(result2, ensure_ascii=False, indent=4))

# Запрос 3: Первые 10 записей из определённого города и профессий, отсортированных по age
city = "Москва"  # Пример произвольного города
jobs = ["Врач", "Программист", "Инженер"]  # Пример профессий
result3 = list(
    collection.find({"city": city, "job": {"$in": jobs}}, {"_id": 0})
    .sort("age", 1)
    .limit(10)
)
df = pd.DataFrame.from_dict(result3)
df.to_json("1_3.json", orient="records", indent=1, default_handler=str, force_ascii=False)
print("\nЗапрос 3:", json.dumps(result3, ensure_ascii=False, indent=4))

# Запрос 4: Количество записей по сложному предикату
age_range = {"$gte": "20", "$lte": "40"}  # Пример диапазона возраста
year_range = ["2019", "2022"]  # Пример диапазона года
salary_conditions = {
    "$or": [
        {"salary": {"$gt": "50000", "$lte": "75000"}},
        {"salary": {"$gt": "125000", "$lt": "150000"}},
    ]
}
result4_count = collection.count_documents(
    {"age": age_range, "year": {"$in": year_range}, **salary_conditions}
)
print("\nЗапрос 4: Количество записей:", result4_count)


Запрос 1: [
    {
        "job": "IT-специалист",
        "salary": "99905",
        "id": "6375180",
        "city": "Ла-Корунья",
        "year": "2022",
        "age": "27"
    },
    {
        "job": "Учитель",
        "salary": "99624",
        "id": "5918289",
        "city": "Кордова",
        "year": "2022",
        "age": "36"
    },
    {
        "job": "Продавец",
        "salary": "99593",
        "id": "1187967",
        "city": "Картахена",
        "year": "2017",
        "age": "56"
    },
    {
        "job": "Программист",
        "salary": "99244",
        "id": "5922874",
        "city": "Валенсия",
        "year": "2016",
        "age": "63"
    },
    {
        "job": "Строитель",
        "salary": "98886",
        "id": "4553345",
        "city": "Луго",
        "year": "2000",
        "age": "63"
    },
    {
        "job": "Косметолог",
        "salary": "98225",
        "id": "5593535",
        "city": "Бургос",
        "year": "2018",
        "age": "30"
    

Задание №2

In [52]:
import pandas as pd
from pymongo import MongoClient
import json

# Подключение к MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["task_2"]
collection = db["task_2"]

records = json.load(open("task_2_item.json", "rb"))

collection.insert_many(records)

# Вывод минимальной, средней, максимальной salary
salary_stats = [
    {
        "$group": {
            "_id": None,
            "min_salary": {"$min": "$salary"},
            "avg_salary": {"$avg": "$salary"},
            "max_salary": {"$max": "$salary"},
        }
    }
]


# Вывод количества данных по представленным профессиям
job_count = [{"$group": {"_id": "$job", "count": {"$sum": 1}}}]


# Вывод минимальной, средней, максимальной salary по городу
city_salary_stats = [
    {
        "$group": {
            "_id": "$city",
            "min_salary": {"$min": "$salary"},
            "avg_salary": {"$avg": "$salary"},
            "max_salary": {"$max": "$salary"},
        }
    }
]


# Вывод минимальной, средней, максимальной salary по профессии
job_salary_stats = [
    {
        "$group": {
            "_id": "$job",
            "min_salary": {"$min": "$salary"},
            "avg_salary": {"$avg": "$salary"},
            "max_salary": {"$max": "$salary"},
        }
    }
]


# Вывод минимального, среднего, максимального возраста по городу
city_age_stats = [
    {
        "$group": {
            "_id": "$city",
            "min_age": {"$min": "$age"},
            "avg_age": {"$avg": "$age"},
            "max_age": {"$max": "$age"},
        }
    }
]


# Вывод минимального, среднего, максимального возраста по профессии
job_age_stats = [
    {
        "$group": {
            "_id": "$job",
            "min_age": {"$min": "$age"},
            "avg_age": {"$avg": "$age"},
            "max_age": {"$max": "$age"},
        }
    }
]


# Вывод максимальной заработной платы при минимальном возрасте
max_salary_min_age = [
    {"$sort": {"age": 1, "salary": -1}},
    {"$limit": 1},
    {"$project": {"_id": 0, "salary": 1, "age": 1}},
]

# Вывод минимальной заработной платы при максимальном возрасте
min_salary_max_age = [
    {"$sort": {"age": -1, "salary": 1}},
    {"$limit": 1},
    {"$project": {"_id": 0, "salary": 1, "age": 1}},
]


# Вывод минимального, среднего, максимального возраста по городу, при условии, что заработная плата больше 50 000, отсортировать вывод по убыванию по полю avg
city_age_stats_filtered = [
    {"$match": {"salary": {"$gt": 50000}}},
    {
        "$group": {
            "_id": "$city",
            "min_age": {"$min": "$age"},
            "avg_age": {"$avg": "$age"},
            "max_age": {"$max": "$age"},
        }
    },
    {"$sort": {"avg_age": -1}},
]
# Вывод минимальной, средней, максимальной salary в произвольно заданных диапазонах по городу, профессии, и возрасту: 18<age<25 || 50<age<65
salary_stats_age_range = [
    {
        "$match": {
            "$or": [
                {"age": {"$gt": 18, "$lt": 25}},
                {"age": {"$gt": 50, "$lt": 65}},
            ]
        }
    },
    {
        "$group": {
            "_id": {"city": "$city", "job": "$job"},
            "min_salary": {"$min": "$salary"},
            "avg_salary": {"$avg": "$salary"},
            "max_salary": {"$max": "$salary"},
        }
    },
]

# Произвольный запрос с $match, $group, $sort
custom_query = [
    {"$match": {"city": "Вильявисиоса"}},
    {"$group": {"_id": "$job", "total_salary": {"$sum": "$salary"}}},
    {"$sort": {"total_salary": -1}},
]


for i, query in enumerate(
    [
        salary_stats,
        job_count,
        city_salary_stats,
        job_salary_stats,
        city_age_stats,
        job_age_stats,
        max_salary_min_age,
        min_salary_max_age,
        city_age_stats_filtered,
        salary_stats_age_range,
        custom_query,
    ]
):
    result = list(collection.aggregate(query))
    print(f"2 Запрос {i+1}: {result}")
    df = pd.DataFrame.from_dict(result)
    df.to_json(
        f"2_{i+1}.json",
        orient="records",
        indent=1,
        default_handler=str,
        force_ascii=False,
    )

2 Запрос 1: [{'_id': None, 'min_salary': 20127, 'avg_salary': 111221.86486486487, 'max_salary': 199543}]
2 Запрос 2: [{'_id': 'IT-специалист', 'count': 627}, {'_id': 'Строитель', 'count': 539}, {'_id': 'Архитектор', 'count': 385}, {'_id': 'Повар', 'count': 473}, {'_id': 'Косметолог', 'count': 363}, {'_id': 'Водитель', 'count': 506}, {'_id': 'Продавец', 'count': 528}, {'_id': 'Оператор call-центра', 'count': 473}, {'_id': 'Программист', 'count': 649}, {'_id': 'Учитель', 'count': 396}, {'_id': 'Психолог', 'count': 462}, {'_id': 'Менеджер', 'count': 440}, {'_id': 'Врач', 'count': 451}, {'_id': 'Инженер', 'count': 660}, {'_id': 'Бухгалтер', 'count': 374}, {'_id': 'Медсестра', 'count': 407}]
2 Запрос 3: [{'_id': 'Загреб', 'min_salary': 58912, 'avg_salary': 122083.13333333333, 'max_salary': 181860}, {'_id': 'Навалькарнеро', 'min_salary': 37913, 'avg_salary': 130095.28571428571, 'max_salary': 191509}, {'_id': 'Картахена', 'min_salary': 61484, 'avg_salary': 149884.36363636365, 'max_salary': 18

Задание №3

In [25]:
from pymongo import MongoClient
import pandas as pd

client = MongoClient("mongodb://localhost:27017/")
db = client["task_3"]
collection = db["task_3"]

data = pd.read_csv("task_3_item.csv", delimiter=";", encoding="utf-8").to_dict(
    orient="records"
)

collection.insert_many(data)

# Удалить из коллекции документы по предикату: salary < 25 000 || salary > 175000
collection.delete_many({"$or": [{"salary": {"$lt": 25000}}, {"salary": {"$gt": 175000}}]})

# Увеличить возраст (age) всех документов на 1
collection.update_many({}, {"$inc": {"age": 1}})

# Поднять заработную плату на 5% для произвольно выбранных профессий
selected_jobs = ["Учитель", "Программист"]
collection.update_many({"job": {"$in": selected_jobs}}, {"$mul": {"salary": 1.05}})

# Поднять заработную плату на 7% для произвольно выбранных городов
selected_cities = ["Москва", "Санкт-Петербург"]
collection.update_many({"city": {"$in": selected_cities}}, {"$mul": {"salary": 1.07}})

# Поднять заработную плату на 10% для выборки по сложному предикату
complex_predicate = {"city": "Москва", "job": {"$in": ["Врач", "Инженер"]}, "age": {"$gte": 30, "$lte": 40}}
collection.update_many(complex_predicate, {"$mul": {"salary": 1.10}})

# Удалить из коллекции записи по произвольному предикату
collection.delete_many({"city": "Тбилиси"})

DeleteResult({'n': 7, 'ok': 1.0}, acknowledged=True)

Задание №4

In [50]:
import pandas as pd
from pymongo import MongoClient
import json
from datetime import datetime


def print_and_save(r, i):
    df = pd.DataFrame.from_dict(r)
    df.to_json(
        f"4_{i}.json",
        orient="records",
        indent=1,
        default_handler=str,
        force_ascii=False,
    )
    print(json.dumps(r, ensure_ascii=False, indent=4))


# Подключение к MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["library"]
authors_collection = db["authors"]
books_collection = db["books"]
genres_collection = db["genres"]

# Чтение данных из файлов
authors_data = pd.read_csv("authors.csv", delimiter=",", encoding="utf-8").to_dict(
    orient="records"
)
books_data = pd.read_csv("books.csv", delimiter=",", encoding="utf-8").to_dict(
    orient="records"
)
genres_data = pd.read_csv("genres.csv", delimiter=",", encoding="utf-8").to_dict(
    orient="records"
)

# Запись данных в MongoDB
authors_collection.insert_many(authors_data)
books_collection.insert_many(books_data)
genres_collection.insert_many(genres_data)

print("Данные успешно записаны в MongoDB")

# Задание 1: Выборка
# 1. Найти всех авторов, родившихся в диапазоне age_range
# Пример диапазона возраста
age_range = {"$gte": 30, "$lte": 50}
authors_in_age_range = list(authors_collection.find({"age": age_range}, {"_id": 0}))
print_and_save(authors_in_age_range, 1)

# 2. Найти все книги определенного жанра
genre_id = 1  # Пример жанра
books_in_genre = list(books_collection.find({"genre_id": genre_id}, {"_id": 0}))
print_and_save(books_in_genre, 2)

# 3. Найти все книги, опубликованные в определенном году
year_published = 1997  # Пример года
books_in_year = list(
    books_collection.find({"year_published": year_published}, {"_id": 0})
)
print_and_save(books_in_year, 3)

# 4. Найти все книги определенного автора
author_id = 32  # Пример автора
books_by_author = list(books_collection.find({"author_id": author_id}, {"_id": 0}))
print_and_save(books_by_author, 4)

# 5. Найти все книги, опубликованные в определенном диапазоне лет
year_range = {"$gte": 1990, "$lte": 2000}  # Пример диапазона лет
books_in_year_range = list(
    books_collection.find({"year_published": year_range}, {"_id": 0})
)
print_and_save(books_in_year_range, 5)

# Задание 2: Агрегация
# 1. Найти средний возраст авторов по жанрам
avg_age_by_genre = list(
    books_collection.aggregate([
        {"$lookup": {"from": "authors", "localField": "author_id", "foreignField": "author_id", "as": "author"}},
        {"$unwind": "$author"},
        {"$group": {"_id": "$genre_id", "avg_age": {"$avg": "$author.age"}}}
    ])
)
print_and_save(avg_age_by_genre, 6)

# 2. Найти количество книг по жанрам
book_count_by_genre = list(
    books_collection.aggregate([
        {"$group": {"_id": "$genre_id", "count": {"$sum": 1}}}
    ])
)
print_and_save(book_count_by_genre, 7)

# 3. Найти количество книг по авторам
book_count_by_author = list(
    books_collection.aggregate([
        {"$group": {"_id": "$author_id", "count": {"$sum": 1}}}
    ])
)
print_and_save(book_count_by_author, 8)

# 4. Найти количество книг по годам публикации
book_count_by_year = list(
    books_collection.aggregate([
        {"$group": {"_id": "$year_published", "count": {"$sum": 1}}}
    ])
)
print_and_save(book_count_by_year, 9)

# 5. Найти количество авторов по возрастным группам
author_count_by_age_group = list(
    authors_collection.aggregate([
        {"$bucket": {"groupBy": "$age", "boundaries": [20, 30, 40, 50, 60, 70, 80, 90], "default": "90+", "output": {"count": {"$sum": 1}}}}
    ])
)
print_and_save(author_count_by_age_group, 9)

# Задание 3: Обновление/удаление данных
# 1. Удалить книги, опубликованные до 1950 года
books_collection.delete_many({"year_published": {"$lt": 1950}})
print("Книги, опубликованные до 1950 года, удалены")

# 2. Удалить авторов, родившихся до 1900 года
authors_collection.delete_many({"birth_year": {"$lt": 1900}})
print("Авторы, родившиеся до 1900 года, удалены")

# 3. Добавление возраста авторам
current_date = datetime.now()
for author in authors_data:
    birth_date = datetime.strptime(author["birth_date"], "%Y-%m-%d")
    age = (
        current_date.year
        - birth_date.year
        - ((current_date.month, current_date.day) < (birth_date.month, birth_date.day))
    )
    authors_collection.update_one(
        {"author_id": author["author_id"]}, {"$set": {"age": age}}
    )

print("Возраст успешно добавлен авторам")

# 4. Увеличить возраст (age) всех авторов на 1
authors_collection.update_many({}, {"$inc": {"age": 1}})
print("Возраст всех авторов увеличен на 1 год")

# 5. Удаление жанра Adventure
genres_collection.delete_one({"genre_name": "Adventure"})
print("Жанр 'Adventure' удален")

Данные успешно записаны в MongoDB
[
    {
        "author_id": 35,
        "author_name": "Zadie Smith",
        "birth_date": "1975-10-25",
        "age": 50
    },
    {
        "author_id": 75,
        "author_name": "Markus Zusak",
        "birth_date": "1975-06-23",
        "age": 50
    },
    {
        "author_id": 35,
        "author_name": "Zadie Smith",
        "birth_date": "1975-10-25",
        "age": 47
    },
    {
        "author_id": 75,
        "author_name": "Markus Zusak",
        "birth_date": "1975-06-23",
        "age": 47
    },
    {
        "author_id": 96,
        "author_name": "N.K. Jemisin",
        "birth_date": "1972-09-19",
        "age": 50
    },
    {
        "author_id": 35,
        "author_name": "Zadie Smith",
        "birth_date": "1975-10-25",
        "age": 46
    },
    {
        "author_id": 75,
        "author_name": "Markus Zusak",
        "birth_date": "1975-06-23",
        "age": 46
    },
    {
        "author_id": 96,
        "author_nam